In [1]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np

# internal imports
from tgb.utils.utils import *
from tgb.linkproppred.evaluate import Evaluator
# from modules.decoder import LinkPredictorTGNPL
# from modules.emb_module import GraphAttentionEmbedding
# from modules.msg_func import TGNPLMessage
# from modules.msg_agg import MeanAggregator
# from modules.memory_module import TGNPLMemory, StaticMemory
from modules.hyper_edgebank import HyperEdgeBankPredictor
from modules.neighbor_loader import LastNeighborLoaderGraphmixer
from modules.early_stopping import  EarlyStopMonitor
from test_hyper_edgebank import test_edgebank
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset, PyGLinkPropPredDatasetHyper
from graphmixer import *

from scipy.stats import pearsonr

/lfs/local/0/zhiyinl/anaconda3/lib/python3.9/site-packages/torch_geometric/typing.py:99: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.27' not found (required by /lfs/turing1/0/zhiyinl/anaconda3/lib/python3.9/site-packages/torch_spline_conv/_basis_cuda.so)
  warnings.warn(


# Initialize data and EdgeBank

In [2]:
# recreate args
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
args = Namespace(dataset='tgbl-hypergraph',
                 lr=1e-4,
                 bs=2000,  # use larger batch size since we're not training
                 k_value=10,
                 num_epoch=100,
                 seed=1,
                 mem_dim=100,
                 time_dim=10,
                 emb_dim=100,
                 tolerance=1e-6,
                 patience=100,
                 num_run=1,
                 wandb=False,
                 bipartite=False,
                 memory_name='static',
                 emb_name='sum',
                 use_inventory=False,
                 debt_penalty=0,
                 consum_rwd=0,
                 gpu=0,
                 num_train_days=-1,
                 num_layers=2,
                 dropout=0.1, 
                 num_neighbors=300,
                 use_prev_sampling=True, # since we tgbl-hypergraph dataset is before change to sampling method
                 time_gap=2000)
device = torch.device("cpu")

In [3]:
# Initialize dataset - copied from run_experiment() in tgnpl.py
with open(os.path.join(PATH_TO_DATASETS, f"{args.dataset.replace('-', '_')}/{args.dataset}_meta.json"), "r") as f:
    metadata = json.load(f)
# set global data variables
num_nodes = len(metadata["id2entity"])  
num_firms = metadata["product_threshold"]
num_products = num_nodes - num_firms              
dataset = PyGLinkPropPredDatasetHyper(name=args.dataset, root="datasets")
metric = dataset.eval_metric
neg_sampler = dataset.negative_sampler
evaluator = Evaluator(name=args.dataset)
data = dataset.get_TemporalData().to(device)
train_loader, val_loader, test_loader = set_up_data(args, data, dataset)

dataset.load_val_ns()  # load validation negative samples

Dataset tgbl-hypergraph url not found, download not supported yet.
file found, skipping download
Dataset directory is  /lfs/turing1/0/zhiyinl/supply-chains/TGB/tgb/datasets/tgbl_hypergraph
loading processed file
Train: N=214674, 273 days; Val: N=44991, 45 days; Test: N=45612, 47 days


In [5]:
# initialize edgebank, fit on train
edgebank = HyperEdgeBankPredictor(num_firms, num_products, consecutive=True)
train_data = data[dataset.train_mask]
edgebank.fit(train_data.src, train_data.dst, train_data.prod)

Fit on 214674 edges; found 38756 unique


In [5]:
test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=False, use_prev_sampling=args.use_prev_sampling)

  0%|                                                                                                                            | 0/23 [00:00<?, ?it/s]/lfs/turing1/0/zhiyinl/supply-chains/TGB/examples/linkproppred/general/test_hyper_edgebank.py:62: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  neg_batch_list = torch.Tensor(neg_batch_list)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 25.75it/s]


0.3399311602115631

In [6]:
test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=True, use_prev_sampling=args.use_prev_sampling)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:03<00:00,  5.76it/s]


0.6819248199462891

In [7]:
# try with subset of train
train_days = data.t[dataset.train_mask].unique()  # original set of train days
days_to_keep = train_days[-30:]  # keep train days from the end, since val follows train
new_train_mask = torch.isin(data.t, days_to_keep)
print(new_train_mask.sum())
edgebank.fit(data[new_train_mask].src, data[new_train_mask].dst, data[new_train_mask].prod)

tensor(30157)
Fit on 30157 edges; found 12434 unique


In [8]:
exist_baseline = test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=False, use_prev_sampling=args.use_prev_sampling)
count_baseline = test_edgebank(val_loader, neg_sampler, "val", evaluator, metric, edgebank, use_counts=True, use_prev_sampling=args.use_prev_sampling)
print(exist_baseline, count_baseline)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:03<00:00,  6.18it/s]

0.4436224400997162 0.6085948348045349


# Try fitting model on tiny data

In [9]:
# first 100 data points in val, NO neighbors
mini_loader = TemporalDataLoader(data[dataset.val_mask][:100], batch_size=args.bs)
assert len(mini_loader) == 1
batch = next(iter(mini_loader))
pos_src, pos_prod, pos_dst, pos_t, pos_msg = batch.src, batch.prod, batch.dst, batch.t, batch.msg
print(pos_src.shape)

torch.Size([100])


In [16]:
# try graphmixer
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on FIXED negative samples from neg_sampler
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, 
                       opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       neg_sampler=neg_sampler, split_mode="val", 
                       num_firms=num_firms, num_products=num_products, use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.23s/it]


10 2.827082872390747


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.10s/it]


20 2.426541328430176


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.06s/it]


...

270 2.0676143169403076


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.27s/it]


280 2.067572593688965


  0%|                                                                                                                             | 0/1 [00:09<?, ?it/s]


KeyboardInterrupt: 

In [17]:
# test MRR on same data
# tgnpl: if we don't reset neighbor loader, results will be bad. If reset, perfect result. 
# graphmixer: reset doesn't seem to affect result
test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products, 
     use_prev_sampling=args.use_prev_sampling)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.64s/it]


0.8283332586288452

In [19]:
# same results once we reset
neighbor_loader.reset_state()
test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
     use_prev_sampling=args.use_prev_sampling)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.51s/it]


0.8283332586288452

In [10]:
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on RANDOM negative samples
# should be able to learn but loss will go down a bit more randomly
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, 
                       opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       ns_samples=6, num_firms=num_firms, num_products=num_products)
    if epoch % 10 == 0:
        print(epoch, loss)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.45s/it]


10 2.8560473918914795


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.46s/it]


20 2.5597734451293945


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.48s/it]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.66s/it]


980 2.0588057041168213


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.10s/it]


990 2.0536205768585205


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:20<00:00, 20.51s/it]


In [11]:
# test MRR on same data
# if we don't reset neighbor loader, results will be bad
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products, 
            use_prev_sampling=args.use_prev_sampling)

# much better results once we reset
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.07s/it]

MRR: before resetting = 0.7982; after resetting = 0.8110


In [4]:
# first 100 data points in val, WITH time-varying neighbors
mini_loader = TemporalDataLoader(data[dataset.val_mask][:100], batch_size=10)
print(len(mini_loader))
batch = next(iter(mini_loader))
pos_src, pos_prod, pos_dst, pos_t, pos_msg = batch.src, batch.prod, batch.dst, batch.t, batch.msg
print(pos_src.shape)

10
torch.Size([10])


In [ ]:
# still trying model with static memory
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on FIXED negative samples from neg_sampler, time-varying neighbors
# results: loss seems a lot more unstable with time-varying neighbors
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       neg_sampler=neg_sampler, split_mode="val", 
                       num_firms=num_firms, num_products=num_products,
                       use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

  0%|                                                                                                          | 0/10 [00:00<?, ?it/s]/lfs/turing1/0/zhiyinl/supply-chains/TGB/examples/linkproppred/general/graphmixer.py:110: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  neg_batch_list = torch.Tensor(neg_batch_list)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.64s/it]


10 2.171579670906067


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.68s/it]


20 2.0956890106201174


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.70s/it]


30 2.0788419008255006


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.63s/it]


40 2.073590087890625


  0%|                                                                                                          | 0/10 [00:00<?, ?it/s]

In [ ]:
# test MRR on same data
# if we don't reset neighbor loader, results will be bad
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

# even with unstable loss, MRR is quite high!
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')

In [ ]:
# still trying model with static memory
model, opt = set_up_model(args, data, device, num_firms=num_firms, num_products=num_products)
neighbor_loader = LastNeighborLoaderGraphmixer(num_nodes, size=args.num_neighbors, edge_feat_dim=1) # size>=num_nodes so time_gap shouldn't be a problem

# fit model on RANDOM negative samples, time-varying neighbors
# should be able to learn but loss will go down a bit more randomly
for epoch in range(1, 1000):
    loss, logits_loss, inv_loss, debt_loss, consump_rwd_loss, update_loss = train(model, opt, neighbor_loader, data, mini_loader, device, 
                       loss_name='ce-softmax', update_params=True, 
                       ns_samples=6, num_firms=num_firms, num_products=num_products,
                       use_prev_sampling=args.use_prev_sampling)
    if epoch % 10 == 0:
        print(epoch, loss)

In [ ]:
# test MRR on same data
# low cross-entropy loss should correspond to high MRR
# if we don't reset neighbor loader, results will be bad
mrr1 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

# results: even with unstable loss, MRR is quite high!
neighbor_loader.reset_state()
mrr2 = test(model, neighbor_loader, data, mini_loader, neg_sampler, evaluator, device,
     split_mode="val", metric="mrr", num_firms=num_firms, num_products=num_products,
           use_prev_sampling=args.use_prev_sampling)

print(f'MRR: before resetting = {mrr1:.4f}; after resetting = {mrr2:.4f}')